# Régression Linéaire

Modèle linéaire pour prédire le prix d'une voiture en fonction de son kilométrage.

## Définitions :

- **Normalisation** : Redimensionner les variables numériques pour qu'elles soient comparables sur une échelle commune.
- **Machine Learning** : Donner à une machine la capacité d'apprendre sans la programmer de façon explicite.
- **Apprentissage supervisé** : Technique d'apprentissage la plus courante en machine learning. On donne des exemples à la machine qu'elle doit étudier pour en créer des modèles.
- **Dataset** : Ensemble d'exemples données à la machine pour apprendre (tableau de données).
- **Problème de regression** : On cherche à prédire la valeur d'une variable continue, c'est-à-dire une variable qui peux prendre une infinité de valeurs.
- **Problème de classification** : On cherche à prédire la valeur d'une variable discrète, c'est-à-dire une variable qui prends certaines valeurs.

## Prérequis :

Pour travailler de manière plus optimale (pour faciliter les calculs notamment), l'utilisation de matrice est fortement recommandée.

In [1]:
from sys import exit
from typing import List, Tuple, Union


Numbers = Union[int, float]

class Matrix:
    __list: List[List[Numbers]]
    __rows: int
    __cols: int

    def __init__(self, matrix: List[List[Numbers]]) -> None:
        self.__list = matrix
        self.__rows = 0
        self.__cols = 0
        if len(matrix) != 0:
            self._validate()

    def _validate(self) -> None:
        self.__rows = len(self.__list)
        self.__cols = len(self.__list[0])
        if not all([len(row) == self.__cols for row in self.__list]):
            exit("All rows in the matrix must have equal number of columns.")

    def __str__(self) -> str:
        separator: str = "-" * 9
        if self.__cols == 0:
            return f"{separator}\n{separator}\n{self.shape()}\n"
        matrix_str: str = "\n".join(str(row) for row in self.__list)
        return f"{separator}\n{matrix_str}\n{separator}\n{self.shape()}\n"

    def _zeroed_matrix(self, sizes: Union[Tuple[int, int], None] = None) -> "Matrix":
        if isinstance(sizes, Tuple):
            return Matrix([[0] * sizes[1] for _ in range(sizes[0])])
        else:
            return Matrix([[0] * self.__cols for _ in range(self.__rows)])

    def __add__(self, other: "Matrix") -> "Matrix":
        if self.shape() != other.shape():
            exit("Both matrices must have the same dimensions.")
        matrix: Matrix = self._zeroed_matrix()
        for row in range(self.__rows):
            for column in range(self.__cols):
                matrix.__list[row][column] = self.__list[row][column] + other.__list[row][column]
        return matrix

    def __sub__(self, other: "Matrix") -> "Matrix":
        if self.shape() != other.shape():
            exit("Both matrices must have the same dimensions.")
        matrix: Matrix = self._zeroed_matrix()
        for row in range(self.__rows):
            for column in range(self.__cols):
                matrix.__list[row][column] = self.__list[row][column] - other.__list[row][column]
        return matrix

    def __mul__(self, other: Union["Matrix", Numbers]) -> "Matrix":
        if isinstance(other, (int, float)):
            matrix: Matrix = self._zeroed_matrix()
            for row in range(self.__rows):
                for column in range(self.__cols):
                    matrix.__list[row][column] = self.__list[row][column] * other
            return matrix
        else:
            if self.__cols != other.__rows:
                exit("")
            matrix: Matrix = self._zeroed_matrix((self.__rows, other.__cols))
            for i in range(self.__rows):
                for j in range(other.__cols):
                    for k in range(self.__cols):
                        matrix.__list[i][j] += self.__list[i][k] * other.__list[k][j]
            return matrix

    def __eq__(self, other: "Matrix") -> bool:
        return self.__list == other.__list

    def get(self) -> List[List[Numbers]]:
        return self.__list

    def min(self) -> float:
        return min(min(row) for row in self.__list)

    def max(self) -> float:
        return max(max(row) for row in self.__list)

    def shape(self) -> Tuple[int, int]:
        return self.__rows, self.__cols

    def sum(self) -> Numbers:
        return sum(sum(row) for row in self.__list)

    def mean(self) -> float:
        return self.sum() / (self.__rows * self.__cols)

    def transpose(self) -> "Matrix":
        matrix: Matrix = self._zeroed_matrix((self.__cols, self.__rows))
        for row in range(self.__rows):
            for column in range(self.__cols):
                matrix.__list[column][row] = self.__list[row][column]
        return matrix

    def square(self) -> "Matrix":
        matrix: Matrix = self._zeroed_matrix()
        for row in range(self.__rows):
            for column in range(self.__cols):
                matrix.__list[row][column] = pow(self.__list[row][column], 2)
        return matrix

## Étape 1 : Dataset

La première étape consiste à récupérer les données de notre dataset. On va extraire deux matrices qui correspondent à notre target et nos features.
Dans notre situation, nous avons donc un vecteur target (prix) et une matrice features (kilométrages).

Voici une représentation de nos matrices :

$$
x = \begin{pmatrix} 240000 \cr 139800 \cr \ldots \cr 61789 \cr \end {pmatrix}
$$

$$
y = \begin{pmatrix} 3650 \cr 3800 \cr \ldots \cr 8290 \cr \end {pmatrix}
$$

Par convention, on note $m$ le nombre d'exemple que l'on a dans notre dataset et on note $n$ le nombre de features que l'on a dans notre dataset. Notre matrice target a une taille de $(m, 1)$ et notre matrice features a une taille de $(m, n)$. Avec notre dataset, nous avons donc $m = 24$ et $n = 1$.

Pour ce faire, voici une classe permettant d'extraire les données (à l'initialisation de notre classe) de notre dataset (notre fichier CSV) en Python.

In [2]:
from csv import reader
from sys import exit
from typing import List, Tuple

def reader_csv(path: str) -> Tuple[Matrix, Matrix]:
    target: List[List[float]] = []
    features: List[List[float]] = []

    try:
        with open(path, mode="r", newline="") as file:
            r = reader(file)
            next(r)
            for row in r:
                target.append([float(row[1])])
                features.append([float(row[0])])
        return Matrix(target), Matrix(features)
    except FileNotFoundError:
        exit(f"The file {path} does not exist.")
    except Exception as e:
        exit(f"An error occurred while reading the CSV file: {e}")

![Raw Data Visualisation](images/raw_data.png)
![Dataset representation](images/dataset.png)

## Étape 2 : Modèle

À partir du dataset (et de la représentation graphique présente plus haut), on peux visualiser un nuage de points. Pour ce projet, nous allons utiliser un modèle linéaire, cependant il existe d'autres type de modèle disponible.

Il est important de noter que c'est nous qui décidons de quel modèle la machine doit utiliser et c'est la machine qui doit apprendre les paramètres. Le modèle est une généralisation de l'ensemble des points de notre dataset, un bon modèle est un modèle qui nous donne les plus petites erreurs.

Pour notre modèle linéaire, nous avons donc la formule suivante :

$$
f(x)=ax+b
$$

La formule sous forme matricielle :

$$
F = X \cdot \theta
$$

Avec :

$$
F = \begin {bmatrix} f(x^1)\cr f(x^2)\cr \ldots\cr  f(x^m)\cr\end {bmatrix}
X = \begin {bmatrix} x^1 & 1\cr x^2 & 1\cr \ldots & \ldots\cr x^m & 1\cr\end {bmatrix}
\theta = \begin {bmatrix} a\cr b \end {bmatrix}
$$

Ce modèle a deux paramètres $a$ et $b$ (cf. coefficients polynome). Comme c'est la machine qui doit apprendre les paramètres, nous utiliserons au lancement du programme des paramètres aléatoire.

Voici une fonction permettant de calculer notre model :

In [3]:
def model(x: Union[Matrix, float], theta: Matrix) -> Union[Matrix, float]:
    if isinstance(x, float):
        theta_value = theta.get()
        return (theta_value[0][0] * x) + theta_value[1][0]
    else:
        return x * theta

## Étape 3 : Fonction Coût

La fonction coût (cf. Fonction Quadratique Moyenne) permet de calculer le coût entre le modèle qu'elle est en train de développer et les vraies valeur de target. Trouver le minimum de la fonction coût revient à trouver le meilleur modèle pour notre programme.

La fonction va mesurer la distance entre la prédiction (le point sur notre droite) et la valeur réelle (cf. norme euclidienne), c'est ce que l'on nomme erreur. On va rassembler toutes les erreurs dans une fonction nommé $J$.

Pour notre fonction coût, la formule est la suivate :

$$
J(a, b) = \frac{1}{2m} \displaystyle\sum_{i=1}^m (f(x^i) - y^i)^2
$$

La formule sous forme matricielle :

$$
J(\theta) = \frac{1}{2m} \displaystyle\sum_{i=1}^m (X \cdot \theta - Y)^2
$$

Voici une fonction représentant la fonction coût :

In [4]:
def cost_function(x: Matrix, y: Matrix, theta: Matrix) -> float:
    m: int = x.shape()[0]
    return (1 / (2 * m)) * (model(x, theta) - y).square().sum()

![Cost function](images/cost.png)
![Error model](images/model_error.png)

## Étape 4 : Algorithme de minimisation

L'algorithme de minimisation est une stratégie qui cherche à trouver quels sont les paramètres de notre modèle qui minimise la fonction coût, c'est-à-dire qui minimise l'ensemble de nos erreurs. Pour notre projet, nous utiliserons la déscente de gradient. La déscente de gradient est un algorithme d'optimisation qui converge vers le minimum d'une fonction convexe.

Dans notre situation, la fonction coût à la même allure qu'une fonction carré (car on fait la somme de carré), c'est-à-dire une allure parabolique. Sur cette fonction, on recherche le minimum de $J$ par rapport à $a$. Pour ce faire, on choisi au hasard un point sur la courbe $J$, on va mesurer sa dérivé et on va aller dans la direction de la pente qui déscends. L'hyper-paramètre $\alpha$ correspond à notre vitesse de convergence (aka. learning_rate).

Pour notre déscente de gradient, voici les formules :

$$
a = a - \alpha \frac{\partial J(a, b)}{\partial a}
$$

$$
b = b - \alpha \frac{\partial J(a, b)}{\partial b}
$$

$$
\theta = \theta - \alpha \frac{\partial J(\theta)}{\partial \theta}
$$

Pour calculer les dérivés de $a$ et $b$ par rapport à $J$, on utilise les formules suivantes :

$$
\frac{\partial J(a, b)}{\partial a} = \frac{1}{m} \displaystyle\sum_{i=1}^m x^i(ax^i + b - y^i)
$$

$$
\frac{\partial J(a, b)}{\partial b} = \frac{1}{m} \displaystyle\sum_{i=1}^m 1(ax^i + b - y^i)
$$

$$
\frac{\partial J(\theta)}{\partial \theta} = \frac{1}{m} X^T (X \cdot \theta - Y)
$$

Avec :

$$
X^T = \begin {bmatrix} x^1 & x^2 & x^3 & \ldots & x^m\cr 1 & 1 & 1 & \ldots & 1 \end {bmatrix}
$$

Voici un exemple de code pour la déscente de gradient :

In [ ]:
def gradient(x: Matrix, y: Matrix, theta: Matrix) -> Matrix:
    m: int = x.shape()[0]
    return x.transpose() * (model(x, theta) - y) * (1 / m)

def gradient_descent(x: Matrix, y: Matrix, theta: Matrix, learning_rate: float, iterations: int) -> Matrix:
    for i in range(iterations):
        theta = theta - (gradient(x, y, theta) * learning_rate)
    return theta

![Regressions](images/regression.png)
![Square function](images/squared_function.png)

## Étape 5 : Coéfficient de détermination

Pour calculer la précision de notre modèle (cf. coefficient de determination), on utilise la formule suivante :

$$
R^2 = 1 - \frac{\displaystyle\sum_{i=1}^n (y^i - \overline{y}^i)^2}{\displaystyle\sum_{i=1}^n (y^i - \overline{y})^2}
$$

Où $n$ est le nombre de mesure, $y^i$ est la valeur de la ième valeur, $\overline{y}^i$ est la valeur prédite correspondante et $\overline{y}$ est la moyenne des mesures.

Voici une implémentation :

In [ ]:
def coefficient_determination(x: Matrix, y: Matrix, theta: Matrix) -> float:
    n: int = y.shape()[0]
    predictions: Matrix = model(x, theta)
    means: Matrix = Matrix([[y.mean()] for _ in range(n)])
    return 1 - ((y - predictions).square().sum() / (y - means).square().sum())